In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

import requests
from bs4 import BeautifulSoup
# from sklearn.Logit_model_reloaded import score
# data_test = pd.read_csv("test_notes.csv", sep=',')

In [2]:
# nice, psg,om, lyon, monaco, lille,rennes, mtp
team_list_av = []
x_av = []
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

page = "https://www.transfermarkt.fr/laliga/startseite/wettbewerb/ES1"

pageTree = requests.get(page, headers=headers)
soup = BeautifulSoup(pageTree.content, 'html.parser')
body = soup.html.find('div',{'class':'large-8 columns'}).find_all('div',{'class':'box'})[1].find('div',{'class':'responsive-table'}).div.table.tbody.find_all('tr')
ixo2 = []
for g in body:

    team_list_av.append(g.find('td',{'hauptlink no-border-links show-for-small show-for-pad'}).a.text)
    ixo = g.find_all('td',{'class':'rechts show-for-small show-for-pad nowrap'})[1].text.replace(' mio. €','').replace(',','.')
    

    if 'mrd' in ixo:
        ixo2.append(float(ixo.replace(' mrd. €',''))*1000)
    else:
        ixo2.append(ixo)
        
    x_av.append(ixo2)

In [3]:
value_model_av = pd.DataFrame({'Team':team_list_av,'Value':x_av[0]})
value_model_av.to_csv('value_model_av.csv',index=False)

In [4]:
value_model_af = pd.read_csv('value_model_av.csv',sep=',')

In [5]:
x = value_model_af['Value']
list_y = []
for i in x:
    list_y.append(0.009231867381092976*i + -2.4393319325352447)

In [6]:
value_model = pd.DataFrame({'Team':value_model_af['Team'],'Value':value_model_af['Value'],'trend':list_y})

In [7]:
value_model

,Team,Value,trend
0,Real Madrid,783.50,4.793836
1,Atlético Madrid,755.20,4.532574
2,Barcelone,676.00,3.801410
3,FC Sevilla,430.90,1.538680
4,Real Sociedad,375.50,1.027234
5,Villarreal CF,298.90,0.320073
6,FC Valencia,251.90,-0.113825
7,Real Betis,223.40,-0.376933
8,Athletic,188.30,-0.700971
9,FC Getafe,159.45,-0.967311


In [8]:
0.013651660777911766*2000+ -2.399976874866872

24.903344680956657

In [9]:
# Récupération du modèle de Régression Logistique 'lr'
with open('model_mhsc.pkl', 'rb') as f:
    my_unpickler = pickle.Unpickler(f)
    model_logit = my_unpickler.load()

    y_pred = model_logit.predict_proba(value_model[['trend']].values) 


In [10]:
for value in y_pred[0]:
    print(value)

0.00424088025913294
0.9957591197408671


In [11]:
predictions = [round(value) for value in y_pred[0]]
# print(ytest)
# precision = model_logit.score(value_model[['Value','trend']],predictions)
# print(np.around(precision,decimals=2)*100,'%')

In [12]:
value_model

,Team,Value,trend
0,Real Madrid,783.50,4.793836
1,Atlético Madrid,755.20,4.532574
2,Barcelone,676.00,3.801410
3,FC Sevilla,430.90,1.538680
4,Real Sociedad,375.50,1.027234
5,Villarreal CF,298.90,0.320073
6,FC Valencia,251.90,-0.113825
7,Real Betis,223.40,-0.376933
8,Athletic,188.30,-0.700971
9,FC Getafe,159.45,-0.967311


In [13]:
predict = []
prt_predict = []
for i in y_pred:
    prt_predict.append(i[1])
value_model['pourcentage'] = prt_predict

count = 0
proba = 6
proba_list = []
high_count = 0
for a,k,i in zip(value_model.sort_values(by='pourcentage',ascending=False)['trend'],value_model.sort_values(by='pourcentage',ascending=False)['Team'],value_model.sort_values(by='pourcentage',ascending=False)['pourcentage']):
    if (proba == 6)&(a < 4):
        proba = proba - high_count
    
    proba_list.append(np.around(i*(proba/6),decimals=3))
    print(i,np.around(i*(proba/6),decimals=3),count)
    count = count + 1
    if a > 4:
        high_count = high_count + 1
        continue
    
    if proba < 4:
        if count == 10:
            proba = proba - 1
        else:
            continue
    else:
        proba = proba - 1
        
    
value_model['pourcentage'] = proba_list
        
for i in list(value_model['pourcentage']):
    if i < 0.5:
        predict.append(False)
    elif i > 0.5:
        predict.append(True)
    
#print(len(predict))
#print(len(y_pred[1]))
value_model['predict'] = predict


0.9957591197408671 0.996 0
0.9942063077014801 0.994 1
0.9861790356907902 0.657 2
0.8251997558279714 0.413 3
0.7187274889975244 0.359 4
0.5223456682164138 0.261 5
0.3938134629786958 0.197 6
0.32145658922225073 0.161 7
0.24305764499184127 0.122 8
0.189134781886387 0.095 9
0.1639617070687692 0.055 10
0.12093596977735818 0.04 11
0.10803085001029439 0.036 12
0.10098228359532117 0.034 13
0.09553501329968998 0.032 14
0.0834113284095542 0.028 15
0.07656605996214613 0.026 16
0.06629394046182135 0.022 17
0.06629394046182135 0.022 18
0.063260501969141 0.021 19


In [14]:
print('Prédictions :', value_model['Team'][0],' : ', np.around(y_pred[0][1],decimals=2),' %')
print('Prédictions :', value_model['Team'][1],' : ', np.around(y_pred[1][1],decimals=2),' %')
print('Prédictions :', value_model['Team'][2],' : ', np.around(y_pred[2][1],decimals=2),' %')
print('Prédictions :', value_model['Team'][3],' : ', np.around(y_pred[3][1],decimals=2),' %')
print('Prédictions :', value_model['Team'][4],' : ', np.around(y_pred[4][1],decimals=2),' %')

Prédictions : Real Madrid  :  1.0  %
Prédictions : Atlético Madrid  :  0.99  %
Prédictions : Barcelone  :  0.99  %
Prédictions : FC Sevilla  :  0.83  %
Prédictions : Real Sociedad  :  0.72  %


In [15]:
# for i,e in zip(value_model['Team'],value_model['pourcentage']):
#     print(i,e)
# MHSC 0.29
# PSG 1.0
# OM 0.89
# OL 0.97
# ASM 0.99
# LOSC 0.91
# NICE 0.77
# SRFC 0.83 

In [16]:
value_model

,Team,Value,trend,pourcentage,predict
0,Real Madrid,783.50,4.793836,0.996,True
1,Atlético Madrid,755.20,4.532574,0.994,True
2,Barcelone,676.00,3.801410,0.657,True
3,FC Sevilla,430.90,1.538680,0.413,False
4,Real Sociedad,375.50,1.027234,0.359,False
5,Villarreal CF,298.90,0.320073,0.261,False
6,FC Valencia,251.90,-0.113825,0.197,False
7,Real Betis,223.40,-0.376933,0.161,False
8,Athletic,188.30,-0.700971,0.122,False
9,FC Getafe,159.45,-0.967311,0.095,False


In [17]:
value_model.to_csv('model_result_liga.csv',index=False)